In [ ]:
"""
API calls to OMDb for tv_show data.
Matt Oliver Taylor Appel
"""
import csv
import numpy as np
import matplotlib.pyplot as plt
import omdb
import time
import pandas as pd
import mysql.connector
from config import pword, host, user, api_key
import Project_1_Webscrape_Functions as web

# define omdb client for API calls
client=omdb.OMDBClient(apikey=api_key)

def omdb_call(tv_title, tv_year):
    '''
    takes in a key to the api, a title of a tv show, and the year it was released
    returns all the imdb data from the omdb API
    '''
    int_year=int(tv_year)
    data = client.get(title=tv_title,  #omdb.OMDBC requires a tv title and a release year
                      year=tv_year,
                      tomatoes=False) #tomatoes is a parameter that returns rotten tomatoes data
    return data

def get_tvshow_data(listy,year):
    '''
    takes in a list of tv shows released in the same year, and a key to the api,
    returns a Pandas Data Frame of all the imdb info on the shows
    '''
    results=[]
    for i, show in enumerate(listy):
        print(str(i)+show)
        results.append(omdb_call(show,year))
        time.sleep(.1) #wait 50 milliseconds
    return pd.DataFrame(results)

def ship_2_sql(dflisty):
    '''
    takes in a tv show dataframe then sends the data into AWS tv show database
    first doing the merchendise data
    '''
    cnx = mysql.connector.connect(
    #connect to mysql platform        
        host =host,
        user =user,
        passwd = pword
        )
    cur=cnx.cursor()
    cur.execute('USE tv_merch')
    for i, show in dflisty.iterrows():
        print(i)
    # first pass the show data into sql
#         cur.execute("""INSERT INTO show_data (
#                                     show_id,
#                                     show_name,
#                                     genre,
#                                     imdb_rating,
#                                     imdb_votes,
#                                     metascore,
#                                     released,
#                                     rated
#                                     ) 
#         VALUES ("{}","{}","{}","{}","{}","{}","{}","{}"
#         )""".format(show['imdb_id'],
#                     show['title'],
#                     show['genre'],
#                     show['imdb_rating'],
#                     show['imdb_votes'],
#                     show['metascore'],
#                     show['year'],
#                     show['rated']))
#     then pass the sales data into sql
        cur.execute("""INSERT INTO merch_data (
                                    show_id,
                                    etsy,
                                    ebay,
                                    bonanza,
                                    show_name
                                    ) 
        VALUES ({},{},{},{},{}
        )""".format(show['imdb_id'],
                    show['etsy'],
                    show['ebay'],
                    show['bonanza'],
                    show['title']))
    cnx.commit()
    cur.close()
    cnx.close()
    

    
    

'''
#test code
show_name = str(input('Name: \n'))
show_year = str(input('Year: \n'))

df = pd.DataFrame(omdb_call( api_key, show_name,show_year))
df.head(1)
'''





#brings all programs together, creating cumulative dataframe and saving to SQL


shows_by_year=web.tv_list_makr()
imdb_data_by_year=[]
# for i, year in enumerate(shows_by_year[9:]):
#     imdb_data_by_year.append(get_tvshow_data(year,2019+i))
frames=[                imdb_data_by_year[1],
                        imdb_data_by_year[2],
                        imdb_data_by_year[3],
                        imdb_data_by_year[4],
                        imdb_data_by_year[5],
                        imdb_data_by_year[6],
                        imdb_data_by_year[7],
                        imdb_data_by_year[8],
                        imdb_data_by_year[9],
                        imdb_data_by_year[10]]
show_data=pd.concat(frames)
show_data['year']



#quick look for element in merch data 

data.drop([86],inplace=True) #unwanted column
j=3
print(data['title'][j])
print(bonanza[j]['result'][0]['Results'])
print(ebay[j]['result'][0]['Results'])
print(etsy[j]['result'][0]['Results'])



#import ebay, etsy and bonanza data into show dataframe

result=lambda x:x['result'][0]['Results']
data['ebay']=[result(ebay[i]) for i in range(len(ebay))]
data['etsy']=[result(etsy[i]) for i in range(len(etsy))]
data['bonanza']=[result(bonanza[i]) for i in range(len(bonanza))]


#dropping unwanted columns

drop_me=['awards',
         'actors',
         'box_office',
         'country',
         'director',
         'dvd',
         'language',
         'metascore',
         'plot',
         'poster',
         'production',
         'response',
         'total_seasons',
         'ratings',
         'type',
         'website',
         'writer']
data.drop(drop_me, axis=1,inplace=True)


#save dataframe
'''
data=pd.read_csv('tv_show_data.csv')
'''

# #data exploration tool
# gimme= ['title','ebay','etsy','bonanza','imdb_votes','imdb_rating','year']
# data[gimme].sort_values(['ebay'],ascending=False)[200:250]
    
# #cleaning etsy data
# etsy_filter=lambda x,y,z: np.NaN if (x>100)and(z<=4)and(y<10) else x
# etsy_high_clip=lambda x: 0 if x>40000 else x
# data['etsy2']=[etsy_medium_clip(data['etsy'][i],data['bonanza']
#[i],data['ebay'][i]) for i in range(len(data['etsy']))]

# #change column type
# data['imdb_votes']=data['imdb_votes'].map(lambda x: x.replace(',','')if type(x)==str else x)
# data['imdb_votes']=data['imdb_votes'].astype(dtype='float64',errors='raise')

#scale the review data for comparison
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
temp_scaled = scaler.fit(temp)
scaled=pd.DataFrame(temp_scaled.transform(temp), columns = ['ebay','etsy2','bonanza','imdb_votes'])



#heat plot of correlation
corr_data=scaled[['ebay','etsy2','bonanza','scaled_total','imdb_votes']]
corr_data['imdb_rating']=data['imdb_rating']
pearsoncorr_scaled= corr_data.corr()
sb.heatmap(pearsoncorr_scaled, 
            xticklabels=pearsoncorr_scaled.columns,
            yticklabels=pearsoncorr_scaled.columns,
            cmap='RdBu_r',
            annot=True,
            linewidth=0.5)



# create table with both scaled and normal data including genre data
genres=[]

for i, show in data.iterrows():
    if type(data['genre'][i])==list:
        for j in range(len(data['genre'][i])):
#             data['genre'][i][j]=data['genre'][i][j].strip()
            stuff={'imdb_id':data['imdb_id'][i],
                   'genre':data['genre'][i][j],
                   'rating':data['imdb_rating'][i],
                   'votes':data['imdb_votes'][i],
                   'etsy2':data['etsy2'][i]/102.1,
                   'ebay':data['ebay'][i]/292.4,
                   'unscaled_total':data['unscaled_total'][i],
                   'bonanza':data['bonanza'][i]/4.851,
                   's_etsy2':scaled['etsy2'][i],
                   's_ebay':scaled['ebay'][i],
                   's_bonanza':scaled['bonanza'][i],
                   's_votes':scaled['imdb_votes'][i],
                   'scaled_total':data['scaled_total'][i],
                   'total_div_votes':(data['unscaled_total'][i])/data['imdb_votes'][i]
                  }
            
            genres.append(stuff)
            print(i,j,data['genre'][i][j]) 
    else:
        pass
        
#plot scaled genre data
genres_data=pd.DataFrame(genres)
genre_plot_data=genres_data.groupby(['genre']).mean().sort_values(['scaled_total'],ascending=False)
genre_plot_data.plot.bar(y=['scaled_total','s_votes'])
